In [1]:
import pandas as pd
import codecs
from pandarallel import pandarallel
import datetime
import numpy as np
import feather
from pathlib import Path
import os

pd.options.display.max_columns = 999
pandarallel.initialize(progress_bar=False)


New pandarallel memory created - Size: 2000 MB
Pandarallel will run on 4 workers


In [2]:
pd.options.display.max_columns = 999
pd.options.display.max_rows = 999

In [3]:
with codecs.open('../01_data/variables_browseNotes_select.txt','r') as f:
    variables = [i[:-1] for i in f.readlines()]

In [4]:
%%time

df = pd.read_csv('../01_data/loan.csv', 
                 low_memory=False,
                 usecols=variables,
                 parse_dates=['issue_d','earliest_cr_line'],
                 date_parser=lambda col: pd.to_datetime(col, format="%b-%Y", errors='coerce').date()
                )

CPU times: user 15min 34s, sys: 10.4 s, total: 15min 45s
Wall time: 16min 12s


In [5]:
df.dtypes

id                                            float64
member_id                                     float64
loan_amnt                                       int64
funded_amnt                                     int64
term                                           object
int_rate                                      float64
installment                                   float64
grade                                          object
sub_grade                                      object
emp_title                                      object
emp_length                                     object
home_ownership                                 object
annual_inc                                    float64
issue_d                                datetime64[ns]
loan_status                                    object
url                                           float64
desc                                           object
purpose                                        object
title                       

In [6]:
def include_exclude(col):
    if col <= pd.to_datetime('2016.03'):
        return True
    else:
        return False
def assign_random():
    return np.random.rand()

In [7]:
df = df.assign(include = lambda x: x['issue_d'].parallel_apply(include_exclude))

In [8]:
from collections import Counter
Counter(df.include.tolist())

Counter({False: 1239341, True: 1021327})

In [9]:
np.random.seed(seed=666)
df['random_value'] = np.random.rand(df.shape[0])

In [10]:
default_map = {
    'Fully Paid': 0,
    'Default' : 1,
    'In Grace Period': 0,
    'Does not meet the credit policy. Status:Fully Paid' : 0,
    'Charged Off': 1,
    'Late (31-120 days)' : 1,
    'Current' : 0,
    'Does not meet the credit policy. Status:Charged Off' : 1,
    'Late (16-30 days)' : 0
}

def default_flag(value):
    return default_map[value]

In [11]:
df = df.assign(default = lambda x: x['loan_status'].parallel_apply(default_flag))

In [12]:
df.head()

,id,member_id,loan_amnt,funded_amnt,term,int_rate,installment,grade,sub_grade,emp_title,emp_length,home_ownership,annual_inc,issue_d,loan_status,url,desc,purpose,title,zip_code,addr_state,dti,delinq_2yrs,earliest_cr_line,inq_last_6mths,mths_since_last_delinq,mths_since_last_record,open_acc,pub_rec,revol_bal,revol_util,total_acc,initial_list_status,collections_12_mths_ex_med,mths_since_last_major_derog,application_type,annual_inc_joint,dti_joint,verification_status_joint,acc_now_delinq,tot_coll_amt,tot_cur_bal,open_acc_6m,open_act_il,open_il_12m,open_il_24m,mths_since_rcnt_il,total_bal_il,il_util,open_rv_12m,open_rv_24m,max_bal_bc,all_util,total_rev_hi_lim,inq_fi,total_cu_tl,inq_last_12m,acc_open_past_24mths,avg_cur_bal,bc_open_to_buy,bc_util,chargeoff_within_12_mths,delinq_amnt,mo_sin_old_rev_tl_op,mo_sin_rcnt_rev_tl_op,mo_sin_rcnt_tl,mort_acc,mths_since_recent_bc,mths_since_recent_inq,mths_since_recent_revol_delinq,num_accts_ever_120_pd,num_actv_bc_tl,num_actv_rev_tl,num_bc_sats,num_bc_tl,num_il_tl,num_op_rev_tl,num_rev_accts,num_rev_tl_bal_gt_0,num_sats,num_tl_120dpd_2m,num_tl_30dpd,num_tl_90g_dpd_24m,num_tl_op_past_12m,pct_tl_nvr_dlq,percent_bc_gt_75,pub_rec_bankruptcies,tax_liens,tot_hi_cred_lim,total_bal_ex_mort,total_bc_limit,total_il_high_credit_limit,revol_bal_joint,sec_app_earliest_cr_line,sec_app_inq_last_6mths,sec_app_mort_acc,sec_app_open_acc,sec_app_revol_util,sec_app_open_act_il,sec_app_num_rev_accts,sec_app_chargeoff_within_12_mths,sec_app_collections_12_mths_ex_med,sec_app_mths_since_last_major_derog,disbursement_method,include,random_value,default
0,NaN,NaN,2500,2500,36 months,13.56,84.92,C,C1,Chef,10+ years,RENT,55000.0,2018-12-01,Current,NaN,NaN,debt_consolidation,Debt consolidation,109xx,NY,18.24,0.0,2001-04-01,1.0,NaN,45.0,9.0,1.0,4341,10.3,34.0,w,0.0,NaN,Individual,NaN,NaN,NaN,0.0,0.0,16901.0,2.0,2.0,1.0,2.0,2.0,12560.0,69.0,2.0,7.0,2137.0,28.0,42000.0,1.0,11.0,2.0,9.0,1878.0,34360.0,5.9,0.0,0.0,212.0,1.0,1.0,0.0,1.0,2.0,NaN,0.0,2.0,5.0,3.0,3.0,16.0,7.0,18.0,5.0,9.0,0.0,0.0,0.0,3.0,100.0,0.0,1.0,0.0,60124.0,16901.0,36500.0,18124.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Cash,False,0.700437,0
1,NaN,NaN,30000,30000,60 months,18.94,777.23,D,D2,Postmaster,10+ years,MORTGAGE,90000.0,2018-12-01,Current,NaN,NaN,debt_consolidation,Debt consolidation,713xx,LA,26.52,0.0,1987-06-01,0.0,71.0,75.0,13.0,1.0,12315,24.2,44.0,w,0.0,NaN,Individual,NaN,NaN,NaN,0.0,1208.0,321915.0,4.0,4.0,2.0,3.0,3.0,87153.0,88.0,4.0,5.0,998.0,57.0,50800.0,2.0,15.0,2.0,10.0,24763.0,13761.0,8.3,0.0,0.0,378.0,4.0,3.0,3.0,4.0,4.0,NaN,0.0,2.0,4.0,4.0,9.0,27.0,8.0,14.0,4.0,13.0,0.0,0.0,0.0,6.0,95.0,0.0,1.0,0.0,372872.0,99468.0,15000.0,94072.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Cash,False,0.844187,0
2,NaN,NaN,5000,5000,36 months,17.97,180.69,D,D1,Administrative,6 years,MORTGAGE,59280.0,2018-12-01,Current,NaN,NaN,debt_consolidation,Debt consolidation,490xx,MI,10.51,0.0,2011-04-01,0.0,NaN,NaN,8.0,0.0,4599,19.1,13.0,w,0.0,NaN,Individual,NaN,NaN,NaN,0.0,0.0,110299.0,0.0,1.0,0.0,2.0,14.0,7150.0,72.0,0.0,2.0,0.0,35.0,24100.0,1.0,5.0,0.0,4.0,18383.0,13800.0,0.0,0.0,0.0,92.0,15.0,14.0,2.0,77.0,14.0,NaN,0.0,0.0,3.0,3.0,3.0,4.0,6.0,7.0,3.0,8.0,0.0,0.0,0.0,0.0,100.0,0.0,0.0,0.0,136927.0,11749.0,13800.0,10000.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Cash,False,0.676514,0
3,NaN,NaN,4000,4000,36 months,18.94,146.51,D,D2,IT Supervisor,10+ years,MORTGAGE,92000.0,2018-12-01,Current,NaN,NaN,debt_consolidation,Debt consolidation,985xx,WA,16.74,0.0,2006-02-01,0.0,NaN,NaN,10.0,0.0,5468,78.1,13.0,w,0.0,NaN,Individual,NaN,NaN,NaN,0.0,686.0,305049.0,1.0,5.0,3.0,5.0,5.0,30683.0,68.0,0.0,0.0,3761.0,70.0,7000.0,2.0,4.0,3.0,5.0,30505.0,1239.0,75.2,0.0,0.0,154.0,64.0,5.0,3.0,64.0,5.0,NaN,0.0,1.0,2.0,1.0,2.0,7.0,2.0,3.0,2.0,10.0,0.0,0.0,0.0,3.0,100.0,100.0,0.0,0.0,385183.0,36151.0,5000.0,44984.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Cash,False,0.727858,0
4,NaN,NaN,30000,30000,60 months,16.14,731.78,C,C4,Mechanic,10+ years,MORTGAGE,57250.0,2018-12-01,Cur

Save excluded data

In [13]:
feather.write_dataframe(df.query('include == False').drop('include', axis=1), '../01_data/DS_loans_OUT.feather')


Included data

In [14]:
df_IN = df.query('include == True').drop('include', axis=1)

Drop irrelevant features

In [15]:
df_IN.select_dtypes(include=['object']).describe().T

,count,unique,top,freq
term,1021327,2,36 months,717289
grade,1021327,7,B,294820
sub_grade,1021327,35,B3,64847
emp_title,960873,325103,Teacher,16015
emp_length,967548,11,10+ years,337941
home_ownership,1021327,6,MORTGAGE,510420
loan_status,1021327,9,Fully Paid,749471
desc,126054,124491,,250
purpose,1021327,14,debt_consolidation,600940
title,1010554,63155,Debt consolidation,484444


In [16]:
df_IN.select_dtypes(exclude=['object']).describe().T

,count,mean,std,min,25%,50%,75%,max
id,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
member_id,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
loan_amnt,1021327.0,14864.527424,8496.667933,500.000000,8000.000000,13000.000000,20000.000000,40000.00
funded_amnt,1021327.0,14852.896159,8492.024793,500.000000,8000.000000,13000.000000,20000.000000,40000.00
int_rate,1021327.0,13.144345,4.451256,5.320000,9.750000,12.990000,15.990000,28.99
installment,1021327.0,439.859617,246.668097,4.930000,261.530000,384.450000,577.655000,1536.95
annual_inc,1021323.0,75739.112984,66630.445505,0.000000,46000.000000,65000.000000,90000.000000,9550000.00
url,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
dti,1021308.0,18.319798,8.645885,-1.000000,12.000000,17.790000,24.170000,999.00
delinq_2yrs,1021298.0,0.318246,0.868505,0.000000,0.000000,0.000000,0.000000,39.00


In [17]:
drop_columns_all_nan = ['id','member_id','url','revol_bal_joint','sec_app_inq_last_6mths','sec_app_mort_acc',
                        'sec_app_open_acc','sec_app_revol_util','sec_app_open_act_il','sec_app_num_rev_accts',
                        'sec_app_chargeoff_within_12_mths','sec_app_collections_12_mths_ex_med',
                        'sec_app_mths_since_last_major_derog']

In [18]:
df_IN =df_IN.drop(drop_columns_all_nan, axis=1)

In [19]:
df.shape

(2260668, 107)

In [20]:
df_IN.shape

(1021327, 93)

In [21]:
feather.write_dataframe(df_IN.query('random_value < 0.7'), '../01_data/DS_loans_IN_train.feather')
feather.write_dataframe(df_IN.query('random_value >= 0.7'), '../01_data/DS_loans_IN_test.feather')
